# Build `MESH` Model Setup for the Bow River at Banff

We use the `MESHFlow` Python package to build a `MESH` model setup for the Bow River at Banff:

Let's start by importing the necessary libraries:

In [1]:
# import necessary libraries
import meshflow # version v0.1.0-dev1

import os # python 3.10.2

Now, let's provide necessary information to set up the `MESH` model:

In [2]:
# main work path - modify
work_path = '/home/kasra545/scratch/bb-models/'

# using meshflow==v0.1.0-dev1
# modify each segment to match your settings
config = {
    'riv': os.path.join(work_path, 'geofabric', 'bb_rivers.shp'),
    'cat': os.path.join(work_path, 'geofabric', 'bb_subbasins.shp'),
    'landcover': os.path.join(work_path, 'gistool-outputs', 'bb_model_stats_NA_NALCMS_landcover_2015v2_30m.csv'),
    'forcing_files': os.path.join(work_path, 'easymore-outputs'),
    'forcing_vars': [ # does the variable list, match those of the "agnostic" step?
        "RDRS_v2.1_P_P0_SFC",
        "RDRS_v2.1_P_HU_09944",
        "RDRS_v2.1_P_TT_09944",
        "RDRS_v2.1_P_UVC_09944",
        "RDRS_v2.1_A_PR0_SFC",
        "RDRS_v2.1_P_FB_SFC",
        "RDRS_v2.1_P_FI_SFC",
    ],
    'forcing_units': { # Here, enter RDRS's original variable units
        'RDRS_v2.1_P_P0_SFC': 'millibar',
        'RDRS_v2.1_P_HU_09944': 'kg/kg',
        'RDRS_v2.1_P_TT_09944': 'celsius',
        'RDRS_v2.1_P_UVC_09944': 'knot',
        'RDRS_v2.1_A_PR0_SFC': 'm/hr',
        'RDRS_v2.1_P_FB_SFC': 'W/m^2',
        'RDRS_v2.1_P_FI_SFC': 'W/m^2',
    },
    'forcing_to_units': { # And here, the units that MESH needs to read
         'RDRS_v2.1_P_UVC_09944': 'm/s',
         'RDRS_v2.1_P_FI_SFC': 'W/m^2',
         'RDRS_v2.1_P_FB_SFC': 'W/m^2',
         'RDRS_v2.1_A_PR0_SFC': 'mm/s',
         'RDRS_v2.1_P_P0_SFC': 'pascal',
         'RDRS_v2.1_P_TT_09944': 'kelvin',
         'RDRS_v2.1_P_HU_09944': 'kg/kg',
    },
    'main_id': 'COMID', # what is the main ID of each river segment? Column name in the `cat` Shapefile
    'ds_main_id': 'NextDownID', # what is the downstream segment ID for each river segment? ditto.
    'landcover_classes': { # these are the classes defined for NALCMS-Landsat 2015 dataset. Is this accurate?
        0: 'Unknown',
        1: 'Temperate or sub-polar needleleaf forest',
        2: 'Sub-polar taiga needleleaf forest',
        3: 'Tropical or sub-tropical broadleaf evergreen forest',
        4: 'Tropical or sub-tropical broadleaf deciduous forest',
        5: 'Temperate or sub-polar broadleaf deciduous forest',
        6: 'Mixed forest',
        7: 'Tropical or sub-tropical shrubland',
        8: 'Temperate or sub-polar shrubland',
        9: 'Tropical or sub-tropical grassland',
        10: 'Temperate or sub-polar grassland',
        11: 'Sub-polar or polar shrubland-lichen-moss',
        12: 'Sub-polar or polar grassland-lichen-moss',
        13: 'Sub-polar or polar barren-lichen-moss',
        14: 'Wetland',
        15: 'Cropland',
        16: 'Barren lands',
        17: 'Urban',
        18: 'Water',
        19: 'Snow and Ice',
    },
    'ddb_vars': { # the stuff that MESH needs: slope, river length, etc... Let me know if there is any issues here!
        'slope': 'ChnlSlope',
        'lengthkm': 'ChnlLength',
        'Rank': 'Rank',
        'Next': 'Next',
        'landcover': 'GRU',
        'unitarea': 'GridArea',
        'landcover_names': 'LandUse',
    },
    'ddb_units': {
        'ChnlSlope': 'm/m',
        'ChnlLength': 'km', # is it in km or m? Please check the units of the Shapefile you created!
        'Rank': 'dimensionless',
        'Next': 'dimensionless',
        'GRU': 'dimensionless',
        'GridArea': 'km^2', # what was the unit of the GridArea, or Shape_Area in the `catchments` Shapefile?
        'LandUse': 'dimensionless',
    },
    'ddb_to_units': {
        'ChnlSlope': 'm/m',
        'ChnlLength': 'm', # This is what MESH needs, no need to change.
        'Rank': 'dimensionless',
        'Next': 'dimensionless',
        'GRU': 'dimensionless',
        'GridArea': 'm^2', # This is what MESH needs, no need to change.
        'LandUse': 'dimensionless',
    },
    'ddb_min_values': {
        'ChnlSlope': 1e-10, # in case there are 0s in the `rivers` Shapefile, we need minimums for certain variables
        'ChnlLength': 1e-3,
        'GridArea': 1e-3,
    },
    'gru_dim': 'NGRU', # change to `NGRU` for 'MESH>=r1860', keep for 'MESH<=1860', for example for r1813.
    'hru_dim': 'subbasin',
    'outlet_value': -9999,
}

We can build an "instance" of the workflow class:

In [3]:
exp1 = meshflow.MESHWorkflow(**config)

And, we can run it using:

In [4]:
exp1.run()

/home/kasra545/github-repos/meshflow/src/meshflow/utility/forcing_prep.py:116: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  var = [i for i in ds.dims.keys() if i != 'time']


Once the run is finished, we can checkout the forcing and drainage database file:

In [5]:
exp1.forcing

<xarray.Dataset> Size: 276kB
Dimensions:                (subbasin: 51, time: 96)
Coordinates:
  * time                   (time) datetime64[ns] 768B 1980-01-01T13:00:00 ......
  * subbasin               (subbasin) int32 204B 71032409 71032292 ... 71028585
Data variables:
    RDRS_v2.1_P_P0_SFC     (subbasin, time) float64 39kB 7.594e+04 ... 7.978e+04
    RDRS_v2.1_P_HU_09944   (subbasin, time) float64 39kB 0.001519 ... 0.001096
    RDRS_v2.1_P_TT_09944   (subbasin, time) float64 39kB 259.4 259.2 ... 256.7
    RDRS_v2.1_P_UVC_09944  (subbasin, time) float64 39kB 1.157 0.9603 ... 0.3347
    RDRS_v2.1_A_PR0_SFC    (subbasin, time) float64 39kB 1.435e-06 ... 6.598e-07
    RDRS_v2.1_P_FB_SFC     (subbasin, time) float64 39kB 0.0 0.0 0.0 ... 0.0 0.0
    RDRS_v2.1_P_FI_SFC     (subbasin, time) float64 39kB 238.2 226.3 ... 226.8
    lat                    (subbasin) float64 408B 51.66 51.63 ... 51.18 51.17
    lon                    (subbasin) float64 408B -116.2 -116.1 ... -115.6
    crs                    int64 8B 1
Attributes:
    author:       University of Calgary
    license:      GNU General Public License v3 (or any later version)
    purpose:      Create forcing .nc file for MESH
    Conventions:  CF-1.6
    history:      Created using MESHFlow version v0.1.0.dev1

In [6]:
exp1.ddb

<xarray.Dataset> Size: 11kB
Dimensions:     (subbasin: 51, NGRU: 14)
Coordinates:
  * subbasin    (subbasin) int64 408B 71032409 71032292 ... 71028597 71028585
Dimensions without coordinates: NGRU
Data variables:
    ChnlSlope   (subbasin) float64 408B 0.01669 0.01355 ... 0.02034 0.006318
    ChnlLength  (subbasin) float64 408B 6.012e+03 493.8 ... 4.375e+03 3.08e+03
    Rank        (subbasin) int32 204B 1 2 3 4 5 6 7 8 ... 45 46 47 48 49 50 51
    Next        (subbasin) int32 204B 6 6 8 8 10 10 12 ... 49 49 50 50 51 51 0
    GRU         (subbasin, NGRU) float64 6kB 0.2154 0.0076 0.0004788 ... 0.0 0.0
    GridArea    (subbasin) float64 408B 6.579e+07 2.559e+07 ... 6.241e+06
    LandUse     (NGRU) <U49 3kB 'Temperate or sub-polar needleleaf forest' .....
    lat         (subbasin) float64 408B 51.66 51.63 51.67 ... 51.2 51.18 51.17
    lon         (subbasin) float64 408B -116.2 -116.1 -116.4 ... -115.6 -115.6
    crs         int64 8B 1
Attributes:
    author:       University of Calgary
    license:      GNU General Public License v3 (or any later version)
    purpose:      Create a drainage database .nc file for MESH
    featureType:  point

In [7]:
# create a directory for MESH setup
try:
    os.makedirs('/home/kasra545/scratch/bb-models/MESH-bow-at-banff')
except FileExistsError:
    pass

# saving drainage database and forcing files
exp1.save('/home/kasra545/scratch/bb-models/MESH-bow-at-banff/')

Now that the two important files are built, we can copy a series of setting files into the output path:

In [8]:
!cp -r setting_files/* /home/kasra545/scratch/bb-models/MESH-bow-at-banff/

____

If there is any issue, open a ticket on [MESHFlow's GitHub Webpage](https://github.com/kasra-keshavarz/meshflow/issues). Once this Notebook runs, then have the setting files along with a `MESH` executable all in the `MESH-smm` directory. Please be mindful of the settings, and assure the settings are all accurate to run `MESH` properly.